In [138]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import seaborn as sns
import warnings

In [139]:
train=pd.read_csv('train.csv')
campaign_data=pd.read_csv('campaign_data.csv')
coupon_item_mapping=pd.read_csv('coupon_item_mapping.csv')
customer_demographics=pd.read_csv('customer_demographics.csv')
customer_transaction_data=pd.read_csv('customer_transaction_data.csv')
item_data=pd.read_csv('item_data.csv')
test=pd.read_csv('test.csv')

In [140]:
print ('Train count:'+ str(len(train)))
print ('Test count:'+ str(len(test)))
print ('campaign_data count:'+ str(len(campaign_data)))
print ('coupon_item_mapping count:'+ str(len(coupon_item_mapping)))
print ('customer_demographics count:'+ str(len(customer_demographics)))
print ('customer_transaction_data count:'+ str(len(customer_transaction_data)))
print ('item_data count:'+ str(len(item_data)))

Train count:78369
Test count:50226
campaign_data count:28
coupon_item_mapping count:92663
customer_demographics count:760
customer_transaction_data count:1324566
item_data count:74066


In [178]:
data = pd.concat([train, test], sort=False).reset_index(drop = True)

In [144]:
ltr = len(train)

In [179]:
data = data.merge(campaign_data, on='campaign_id')#  campaign_data
data = data.merge(customer_demographics, on='customer_id',how='left') #  customer_demographics

In [180]:
data = pd.merge_asof(data.sort_values("customer_id"), customer_transaction_data.sort_values("customer_id"), on='customer_id')
data = pd.merge_asof(data.sort_values("item_id"), item_data.sort_values("item_id"), on='item_id')

In [151]:
len(data)

128595

In [181]:
data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,...,income_bracket,date,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,11580,18,956,85,NaN,X,10/08/13,04/10/13,46-55,NaN,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,Local,Grocery
1,23134,18,199,85,NaN,X,10/08/13,04/10/13,46-55,NaN,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,Local,Grocery
2,24558,13,1032,85,0.0,X,19/05/13,05/07/13,46-55,NaN,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,Local,Grocery
3,52925,8,433,85,0.0,X,16/02/13,05/04/13,46-55,NaN,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,Local,Grocery
4,38200,18,702,85,NaN,X,10/08/13,04/10/13,46-55,NaN,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,Local,Grocery


In [192]:
data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,...,income_bracket,date,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,11580,18,956,85,NaN,0,2013-10-08,2013-04-10,0,-1,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,0,0
1,23134,18,199,85,NaN,0,2013-10-08,2013-04-10,0,-1,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,0,0
2,24558,13,1032,85,0.0,0,2013-05-19,2013-05-07,0,-1,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,0,0
3,52925,8,433,85,0.0,0,2013-02-16,2013-05-04,0,-1,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,0,0
4,38200,18,702,85,NaN,0,2013-10-08,2013-04-10,0,-1,...,3.0,2013-03-29,37,2,95.11,-18.17,0.0,56,0,0


In [183]:
data['start_date'] = pd.to_datetime(data['start_date'])
data['end_date'] = pd.to_datetime(data['end_date'])
data['date'] = pd.to_datetime(data['date'])

In [197]:
data['family_size'].unique()

array([ 0, -1,  1,  2,  3,  4], dtype=int64)

In [204]:
data['family_size'].factorize()

(array([0, 0, 0, ..., 1, 1, 1], dtype=int64),
 Int64Index([0, -1, 1, 2, 3, 4], dtype='int64'))

In [203]:
for i in data.columns:
    if str(data[i].dtype) == 'object':
        data[i] = data[i].factorize()[0]

In [205]:
train_cols = ['campaign_id', 'coupon_id', 'customer_id',
       'campaign_type', 'age_range','marital_status', 'rented', 'family_size', 'no_of_children',
       'income_bracket', 'item_id', 'quantity', 'selling_price',
       'other_discount', 'coupon_discount', 'brand', 'brand_type', 'category']

In [206]:
data[train_cols] = data[train_cols].fillna(data[train_cols].mean())

In [207]:
train = data[data['redemption_status'].notnull()]
test = data[data['redemption_status'].isnull()]

In [208]:
data = pd.concat([train, test], sort=False).reset_index(drop = True)
ltr = len(train)

In [210]:
def get_importances(clfs):
    importances = [clf.feature_importance('gain') for clf in clfs]
    importances = np.vstack(importances)
    mean_gain = np.mean(importances, axis=0)
    features = clfs[0].feature_name()
    data = pd.DataFrame({'gain':mean_gain, 'feature':features})
    plt.figure(figsize=(8, 30))
    sns.barplot(x='gain', y='feature', data=data.sort_values('gain', ascending=False))
    plt.tight_layout()
    return data